In [1]:
# !pip install git+https://github.com/google/BIG-bench.git 

In [1]:
from bigbench.benchmark_tasks.code_line_description import task as cld_task




ImportError: cannot import name 'task' from 'bigbench.benchmark_tasks.code_line_description' (unknown location)

In [66]:
from bigbench.benchmark_tasks.boolean_expressions import task as be_task
import random

# we will simply take the tasks, rather than use bigbench's evaluator
# this converts the task into a json task
class ECBooleanExpressions(be_task.BooleanExpressionsTask):
    """Extracting boolean expressions task of variable difficulty."""

    def expressions_for(self, shots=None):
        if shots is None:
            shots = [self.num_shots]
        # TODO: iterate over self.expression_lengths
        expr_size = self.expression_lengths[2]
        expressions = list(self._yield_expression(expr_size))

        for shot in shots:
            # sort and reshuffle for each shot
            expressions = sorted(expressions)
            random.shuffle(expressions)
            # gather expressions in shot+1 windows
            for i in range(0, len(expressions), shot + 1):
                *shot_prompts, question = [
                    self._eval_expression(expr)
                    for expr in expressions[i : i + shot + 1]
                ]
                prompt = "".join(
                    prompt + str(truth) + " . "
                    for (truth, prompt) in shot_prompts
                )
                question_truth, question_prompt = question
                prompt += question_prompt
                print({
                    "prompt": prompt,
                    "truth": question_truth,
                    "shot": shot,
                    "expr_size": expr_size,
                })
            # truth, prompt = self._eval_expression(expressions[0])
            # test_case = {
                # "shots": 1,
            # }

ecbe = ECBooleanExpressions(seed=13)
print(ecbe.expressions_for(shots=[0, 1]))

{'prompt': 'True or False is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': 'True and True is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': '( False ) is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'not not False is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'False or True is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': '( True ) is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': 'True and False is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'True or True is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': 'False and True is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'False and False is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'not not True is ', 'truth': True, 'shot': 0, 'expr_size': 3}
{'prompt': 'False or False is ', 'truth': False, 'shot': 0, 'expr_size': 3}
{'prompt': 'True and True is True . False and True is ', 'truth': False, 'shot': 1, 'expr_size': 3}
{'prompt': '

<class 'bigbench.benchmark_tasks.boolean_expressions.task.BooleanExpressionsTask'>
